In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
from tqdm import tqdm

def extract_all_frames_with_progress(drive_root, output_root, every_n_frames=10):
    categories = ['real', 'fake']
    for category in categories:
        input_folder = os.path.join(drive_root, category)
        output_folder = os.path.join(output_root, category)
        os.makedirs(output_folder, exist_ok=True)

        videos = os.listdir(input_folder)
        print(f"\nProcessing {category.upper()} videos...")
        for video_name in tqdm(videos):
            video_path = os.path.join(input_folder, video_name)
            cap = cv2.VideoCapture(video_path)
            count = 0
            frame_id = 0
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break
                if frame_id % every_n_frames == 0:
                    filename = f"{video_name.replace('.mp4', '')}_frame_{count}.jpg"
                    frame_path = os.path.join(output_folder, filename)
                    cv2.imwrite(frame_path, frame)
                    count += 1
                frame_id += 1
            cap.release()

drive_video_path = "/content/drive/MyDrive/video_batches/batch_1"
local_output_path = "/content/data"
extract_all_frames_with_progress(drive_video_path, local_output_path, every_n_frames=10)


In [ ]:
import tensorflow as tf
import pathlib

data_dir = pathlib.Path('/content/data')
img_size = 128
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_size, img_size),
    batch_size=batch_size
)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.TimeDistributed(layers.Conv2D(32, (3, 3), activation='relu'), input_shape=(1, img_size, img_size, 3)),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Conv2D(64, (3, 3), activation='relu')),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Conv2D(128, (3, 3), activation='relu')),
    layers.TimeDistributed(layers.MaxPooling2D((2, 2))),
    layers.TimeDistributed(layers.Flatten()),
    layers.LSTM(64),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
def expand_dims_for_rnn(dataset):
    return dataset.map(lambda x, y: (tf.expand_dims(x, axis=1), y))

train_rnn = expand_dims_for_rnn(train_ds)
val_rnn = expand_dims_for_rnn(val_ds)

history = model.fit(train_rnn, validation_data=val_rnn, epochs=10)

In [ ]:
model.save('/content/deepfake_cnn_rnn_model.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def predict_frame(frame_path):
    img = tf.keras.preprocessing.image.load_img(frame_path, target_size=(img_size, img_size))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = tf.expand_dims(img_array, axis=0)        
    img_array = tf.expand_dims(img_array, axis=1)        
    pred = model.predict(img_array)
    return "FAKE" if pred[0][0] > 0.5 else "REAL"


In [ ]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy', marker='o')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy', marker='o')
    plt.title('Training and Validation Accuracy')
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss', marker='o')
    plt.plot(epochs_range, val_loss, label='Validation Loss', marker='o')
    plt.title('Training and Validation Loss')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.tight_layout()
    plt.show()

plot_training_history(history)
